# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax

pd.options.display.max_rows = 100

/Users/maria/.local/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
#pip install mlxtend

### Скорректируем данные

In [4]:
data['country'] = data['country'].str.strip(' ')
data['country'] = data['country'].str.capitalize()

In [5]:
data['artist'] = data['artist'].str.strip(' ')
data['artist'] = data['artist'].str.lower()

### Возьмем только Germany, Netherlands

In [6]:
data = data[data['country'].isin(['Germany','Netherlands'])]
# Netherlands Antilles - don't include

In [7]:
data['country'].value_counts()

Germany        24251
Netherlands     9673
Name: country, dtype: int64

### Проверим пустые и неизвестные значения

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33924 entries, 0 to 289615
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     33924 non-null  int64 
 1   artist   33924 non-null  object
 2   sex      33924 non-null  object
 3   country  33924 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [9]:
data['sex'].unique()

array(['f', 'm'], dtype=object)

In [10]:
data['country'].unique()

array(['Germany', 'Netherlands'], dtype=object)

In [11]:
data[(data['artist'] == '[unknown]')].count()

user       95
artist     95
sex        95
country    95
dtype: int64

In [12]:
data = data[(data['artist'] != '[unknown]')]

In [13]:
data[(data['artist'] == '[unknown]')].count()

user       0
artist     0
sex        0
country    0
dtype: int64

### Проверим данные по артистам

In [14]:
data['artist'].value_counts().describe()

count    1001.000000
mean       33.795205
std        35.136747
min         1.000000
25%        13.000000
50%        22.000000
75%        41.000000
max       296.000000
Name: artist, dtype: float64

In [15]:
### Кажется, что артисты с малым количеством слушателей не явлются релевантными
### при поиске паттернов. Возьмем группы со значением больше 10

In [16]:
frequent_artist = data.groupby(['artist']).filter(lambda g: len(g) > 250)

In [17]:
frequent_artist['artist'].unique()

array(['red hot chili peppers', 'coldplay'], dtype=object)

In [18]:
data = data.groupby(['artist']).filter(lambda x: x['artist'].count() > 10.)

In [19]:
data['artist'].value_counts().describe()

count    826.000000
mean      39.480630
std       36.195868
min       11.000000
25%       17.000000
50%       27.000000
75%       47.000000
max      296.000000
Name: artist, dtype: float64

In [20]:
### Проверим данные по пользователям

In [21]:
data['user'].value_counts().describe()

count    1722.000000
mean       18.937863
std        10.127036
min         1.000000
25%        11.000000
50%        19.000000
75%        26.000000
max        53.000000
Name: user, dtype: float64

In [22]:
### Кажется, что пользователи с малым количеством групп не явлются релевантными
### при поиске паттернов. Возьмем пользователей со значением больше 10

In [23]:
data = data.groupby(['user']).filter(lambda x: x['user'].count() >= 3.)

In [24]:
data['user'].value_counts().describe()

count    1671.000000
mean       19.470975
std         9.802002
min         3.000000
25%        11.000000
50%        19.000000
75%        27.000000
max        53.000000
Name: user, dtype: float64

In [25]:
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


# Let's find rules

## By user in two countries

In [26]:
data_by_user = data.groupby(['user', 'country', 'sex'])['artist'].apply(lambda x: ','.join(x).lower().strip()).reset_index()
data_by_user

,user,country,sex,artist
0,1,Germany,f,"red hot chili peppers,the black dahlia murder,..."
1,31,Netherlands,f,"jamiroquai,tori amos,jazzanova,radiohead,st. g..."
2,33,Germany,f,"death cab for cutie,tegan and sara,kimya dawso..."
3,42,Germany,f,"soundtrack,groove coverage,avril lavigne,the r..."
4,51,Germany,f,"kate nash,arctic monkeys,lykke li,adele,tegan ..."
...,...,...,...,...
1666,19662,Germany,m,"blind guardian,in extremo,subway to sally,finn..."
1667,19672,Germany,m,"system of a down,björk,portishead,the prodigy,..."
1668,19681,Netherlands,m,"the strokes,muse,led zeppelin,snow patrol,radi..."
1669,19685,Netherlands,m,"philip glass,rammstein,pink floyd,moby,blank &..."


In [27]:
#dummy encoding
dummy_data_by_user = data_by_user['artist'].str.get_dummies(',')
dummy_data_by_user = pd.concat([data_by_user, dummy_data_by_user], axis = 1)
dummy_data_by_user

,user,country,sex,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,36 crazyfists,44,...,venetian snares,vnv nation,volbeat,weezer,within temptation,wolfgang amadeus mozart,yann tiersen,yeah yeah yeahs,yellowcard,zero 7
0,1,Germany,f,"red hot chili peppers,the black dahlia murder,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,31,Netherlands,f,"jamiroquai,tori amos,jazzanova,radiohead,st. g...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33,Germany,f,"death cab for cutie,tegan and sara,kimya dawso...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42,Germany,f,"soundtrack,groove coverage,avril lavigne,the r...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51,Germany,f,"kate nash,arctic monkeys,lykke li,adele,tegan ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,19662,Germany,m,"blind guardian,in extremo,subway to sally,finn...",0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1667,19672,Germany,m,"system of a down,björk,portishead,the prodigy,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1668,19681,Netherlands,m,"the strokes,muse,led zeppelin,snow patrol,radi...",0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1669,19685,Netherlands,m,"philip glass,rammstein,pink floyd,moby,blank &...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
frequent_itemsets_user = apriori(dummy_data_by_user.drop(columns=['user','country', 'sex', 'artist']), 
                            min_support=0.015, use_colnames=True)

frequent_itemsets_user

,support,itemsets
0,0.018552,(2pac)
1,0.059246,(3 doors down)
2,0.041891,(30 seconds to mars)
3,0.020347,(36 crazyfists)
4,0.020946,(50 cent)
...,...,...
996,0.017355,"(rammstein, koяn, system of a down)"
997,0.015560,"(slipknot, koяn, system of a down)"
998,0.016756,"(rammstein, metallica, system of a down)"
999,0.015560,"(slipknot, rammstein, system of a down)"


In [29]:
rules_by_user = association_rules(frequent_itemsets_user, metric="confidence", min_threshold=0.5)
#rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules_by_user

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(tool),0.029922,0.049671,0.016158,0.540000,10.871566,0.014672,2.065933
1,(anti-flag),(rise against),0.025733,0.072412,0.017355,0.674419,9.313665,0.015491,2.849021
2,(the last shadow puppets),(arctic monkeys),0.025135,0.102932,0.015560,0.619048,6.014120,0.012972,2.354803
3,(jason mraz),(coldplay),0.034111,0.177139,0.017355,0.508772,2.872155,0.011312,1.675109
4,(keane),(coldplay),0.038899,0.177139,0.024536,0.630769,3.560863,0.017646,2.228581
5,(die toten hosen),(die ärzte),0.086774,0.147816,0.049072,0.565517,3.825827,0.036246,1.961377
6,(farin urlaub),(die ärzte),0.060443,0.147816,0.037104,0.613861,4.152884,0.028169,2.206939
7,(equilibrium),(ensiferum),0.029922,0.036505,0.016158,0.540000,14.792459,0.015066,2.094554
8,(good charlotte),(linkin park),0.035907,0.146619,0.017953,0.500000,3.410204,0.012689,1.706762
9,(in extremo),(subway to sally),0.054458,0.057451,0.027528,0.505495,8.798764,0.024400,1.906044


In [30]:
# support - показатель «частотности»
# confidence -  показатель того, как часто наше правило срабатывает для всего датасета.
# lift - 
# leverage - отношение «зависимости» items к их «независимости»
# conviction — это «частотность ошибок»  правила
rules_by_user[ 
       (rules_by_user['antecedent support'] > 0.03) &
       (rules_by_user['lift'] > 7) &
       (rules_by_user['support'] > 0.015) 
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(in extremo),(subway to sally),0.054458,0.057451,0.027528,0.505495,8.798764,0.024400,1.906044
14,(schandmaul),(subway to sally),0.046080,0.057451,0.024536,0.532468,9.268263,0.021889,2.016008
28,"(koяn, system of a down)",(slipknot),0.031119,0.055655,0.015560,0.500000,8.983871,0.013828,1.888689


Видим, что если пользователю нравится металкор, то ему понравятся и другие группы металкор, если pop - то посоветовать pop.

In [31]:
frequent_itemsets_user2 = fpgrowth(dummy_data_by_user.drop(columns=['user','country', 'sex','artist']), 
                            min_support=0.015, use_colnames=True)
frequent_itemsets_user2

,support,itemsets
0,0.159785,(red hot chili peppers)
1,0.108318,(the killers)
2,0.102932,(jack johnson)
3,0.049671,(the rolling stones)
4,0.046080,(schandmaul)
...,...,...
996,0.020946,"(the killers, editors)"
997,0.020347,"(bloc party, editors)"
998,0.016756,"(editors, the kooks)"
999,0.015560,"(editors, interpol)"


In [32]:
rules_by_user2 = association_rules(frequent_itemsets_user2, metric="confidence", min_threshold=0.5)
#rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules_by_user2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(schandmaul),(subway to sally),0.046080,0.057451,0.024536,0.532468,9.268263,0.021889,2.016008
1,"(the killers, arctic monkeys)",(the kooks),0.034111,0.097546,0.017355,0.508772,5.215693,0.014027,1.837138
2,(die toten hosen),(die ärzte),0.086774,0.147816,0.049072,0.565517,3.825827,0.036246,1.961377
3,"(red hot chili peppers, die toten hosen)",(die ärzte),0.031119,0.147816,0.020347,0.653846,4.423388,0.015747,2.461866
4,"(rammstein, die toten hosen)",(die ärzte),0.028127,0.147816,0.017953,0.638298,4.318201,0.013796,2.356039
5,(farin urlaub),(die ärzte),0.060443,0.147816,0.037104,0.613861,4.152884,0.028169,2.206939
6,"(farin urlaub, die toten hosen)",(die ärzte),0.022142,0.147816,0.015560,0.702703,4.753912,0.012287,2.866438
7,(keane),(coldplay),0.038899,0.177139,0.024536,0.630769,3.560863,0.017646,2.228581
8,(three days grace),(linkin park),0.029324,0.146619,0.016756,0.571429,3.897376,0.012457,1.991223
9,(in extremo),(subway to sally),0.054458,0.057451,0.027528,0.505495,8.798764,0.024400,1.906044


In [33]:
rules_by_user2[ 
       (rules_by_user2['antecedent support'] > 0.03) &
       (rules_by_user2['lift'] > 7) &
       (rules_by_user2['support'] > 0.015) 
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(schandmaul),(subway to sally),0.046080,0.057451,0.024536,0.532468,9.268263,0.021889,2.016008
9,(in extremo),(subway to sally),0.054458,0.057451,0.027528,0.505495,8.798764,0.024400,1.906044
16,"(koяn, system of a down)",(slipknot),0.031119,0.055655,0.015560,0.500000,8.983871,0.013828,1.888689


Нет каких-либо нетривиальных пар.

In [199]:
rules = association_rules(frequent_itemsets_user, metric="support", min_threshold=0.015)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

In [233]:
rules[ 
    (rules['conviction'] < 1) 
    ]
# "Антиправила" с lift < 1
# Такие рекомендации делать не стоит

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
382,(nirvana),(coldplay),0.091562,0.177139,0.016158,0.176471,0.996224,-0.000061,0.999188,1
383,(coldplay),(nirvana),0.177139,0.091562,0.016158,0.091216,0.996224,-0.000061,0.999620,1
398,(rammstein),(coldplay),0.136445,0.177139,0.020946,0.153509,0.866599,-0.003224,0.972084,1
399,(coldplay),(rammstein),0.177139,0.136445,0.020946,0.118243,0.866599,-0.003224,0.979357,1


In [275]:
rules[ 
    (rules['confidence'] > 0.2) & 
    (rules['lift'] > 1) &
    (rules['consequents'] == {'maxïmo park'}) 
    ]
# Не самая популярная группа maxïmo park
# Можно для продвижения рекомендовать ее пользователям, слушающим arctic monkeys, bloc party, mando diao, the kooks

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
99,(arctic monkeys),(maxïmo park),0.102932,0.054458,0.026930,0.261628,4.804178,0.021324,1.280576,1
258,(bloc party),(maxïmo park),0.089168,0.054458,0.022741,0.255034,4.683089,0.017885,1.269241,1
836,(mando diao),(maxïmo park),0.067624,0.054458,0.015560,0.230088,4.225032,0.011877,1.228117,1
863,(the kooks),(maxïmo park),0.097546,0.054458,0.021544,0.220859,4.055552,0.016232,1.213569,1
